In [2]:
import json
import pickle
import os
import sys
from tqdm import tqdm

sys.path.insert(0, r'C:\work\DrugDiscovery\main_git\XAI_Chem\ml_part\EAI_for_molecule')
sys.path.insert(0, r'C:\work\DrugDiscovery\main_git\XAI_Chem\ml_part')

import pandas as pd
import torch
from utils import setup, load_model, predict
from lrp import Lrp
from constants import ModelType

In [3]:
def extract_all_smiles_and_fgroups(csv_path):
    smiles_and_fgroup_dict = {}
    df = pd.read_csv(csv_path)
    for value, row in df.iterrows():
        smiles_and_fgroup_dict[str(row['smiles'])] = str(row['f_group'])

    return smiles_and_fgroup_dict


In [4]:
from utils import setup, load_model, predict
from My_Pka_Model import Pka_acidic_view,Pka_basic_view
from dgllife.utils import smiles_to_bigraph
import torch

def load_logP_model(exp_config, args):
    logP_model = load_model(exp_config).to(args['device'])
    # checkpoint = torch.load(r"C:\work\DrugDiscovery\main_git\XAI_Chem\ml_part\weights\logP\model_pretrain_76_default_weights.pth",map_location=torch.device('cpu'))#my_model/CRT76-logD/model_pretrain_76.pth"
    checkpoint = torch.load(r"C:\work\DrugDiscovery\main_git\XAI_Chem\ml_part\weights\logP\logP_RTLogD_best_loss_comfy-wave-9.pth",map_location=torch.device('cpu'))#my_model/CRT76-logD/model_pretrain_76.pth"
    logP_model.load_state_dict(checkpoint)

    return logP_model


def load_pKa_acidic_model(args):
    pka1_model = Pka_acidic_view(node_feat_size = 74,
                                 edge_feat_size = 12,
                                 output_size = 1,
                                 num_layers= 6,
                                 graph_feat_size=200,
                                 dropout=0).to(args['device'])
    # pka1_model.load_state_dict(torch.load(r'C:\work\DrugDiscovery\RT_LogP_with_pKa_model\RTlogD\Trained_model/site_acidic.pkl',map_location='cpu'))
    pka1_model.load_state_dict(torch.load(r'C:\work\DrugDiscovery\main_git\XAI_Chem\ml_part\weights\pKa\site_acidic_best_loss_distinctive-butterfly-11.pkl',map_location='cpu'))

    return pka1_model


def load_pKa_basic_model(args):
    pka2_model = Pka_basic_view(node_feat_size = 74,
                                edge_feat_size = 12,
                                output_size = 1,
                                num_layers= 6,
                                graph_feat_size=200,
                                dropout=0).to(args['device'])
    # pka2_model.load_state_dict(torch.load(r'C:\work\DrugDiscovery\main_git\XAI_Chem\ml_part\weights\pKa\site_basic.pkl',map_location='cpu'))
    pka2_model.load_state_dict(torch.load(r'C:\work\DrugDiscovery\main_git\XAI_Chem\ml_part\weights\pKa\site_amine_best_loss_sweet-capybara-11.pkl',map_location='cpu'))
    return pka2_model

def convert_smiles_to_dglgraph(smiles: str, args):
    dglgraph = smiles_to_bigraph(smiles=smiles,
                                 node_featurizer=args['node_featurizer'],
                                 edge_featurizer=args['edge_featurizer'])

    return dglgraph

In [6]:

pKa_acid_csv_path = r'C:\work\DrugDiscovery\main_git\XAI_Chem\data\FGroupData\test_pKa_amine_data.csv'
output_path = r'C:\work\DrugDiscovery\main_git\XAI_Chem\data\output_EAI\pKa_amine_all_val_smiles'

smiles_and_fgroup_dict = extract_all_smiles_and_fgroups(pKa_acid_csv_path)

if __name__ == '__main__':
    with open(r'C:\work\DrugDiscovery\main_git\XAI_Chem\ml_part\configs\args.pickle', 'rb') as file:
        args =pickle.load(file)
        args['task'] = ['pKa']
    with open(r'C:\work\DrugDiscovery\main_git\XAI_Chem\ml_part\configs\configure.json', 'r') as f:
        exp_config = json.load(f)
    args['device'] = torch.device('cpu')
    args = setup(args)

    logP_model = load_logP_model(exp_config, args)
    amine_model = load_pKa_basic_model(args)
    acid_model = load_pKa_acidic_model(args)

    logP_model.eval()
    amine_model.eval()
    acid_model.eval()

    index = 0
    for smiles, fgroup in tqdm(smiles_and_fgroup_dict.items()):

        bg = convert_smiles_to_dglgraph(smiles, args)
        logP_pred, pKa_acid_pred, pKa_amine_pred = predict(args, logP_model, bg, default_weights=False)
        
        lrp_class = Lrp(logP_model=logP_model, 
                        amine_pKa_model=amine_model,
                        acid_pKa_model=acid_model,
                        smiles=smiles, 
                        functional_group=fgroup, 
                        exp_config=exp_config, 
                        args=args,
                        model_type=ModelType.pKa_amine)

        svg_filename = f'{index}_{smiles}.svg'
        output_svg_path = os.path.join(output_path, svg_filename)
        lrp_class.save_molecule_with_relevances(output_svg_path)
        index += 1
        print(index, smiles, logP_pred, pKa_acid_pred, pKa_amine_pred)

in node features  76


  5%|▌         | 1/20 [00:00<00:18,  1.03it/s]

1 FC1(F)CC(N)CCC1 tensor([[0.0818, 2.3264]], grad_fn=<AddmmBackward0>) tensor([[11.8259]]) tensor([[9.0229]])


 10%|█         | 2/20 [00:02<00:19,  1.08s/it]

2 NC(C1)CC21CC(F)(F)C2 tensor([[0.0225, 2.3818]], grad_fn=<AddmmBackward0>) tensor([[12.1985]]) tensor([[9.6962]])


...

18 FC1(F)CNCCC1 tensor([[0.0547, 1.2576]], grad_fn=<AddmmBackward0>) tensor([[8.2331]]) tensor([[7.3197]])


 95%|█████████▌| 19/20 [00:19<00:01,  1.07s/it]

19 C1(NC2)CC2C1 tensor([[-0.4511,  1.3393]], grad_fn=<AddmmBackward0>) tensor([[11.2090]]) tensor([[10.3349]])


100%|██████████| 20/20 [00:20<00:00,  1.03s/it]

20 NCC([H])(F)F tensor([[0.0607, 1.2681]], grad_fn=<AddmmBackward0>) tensor([[10.9197]]) tensor([[6.9106]])


In [5]:

pKa_acid_csv_path = r'C:\work\DrugDiscovery\main_git\XAI_Chem\data\FGroupData\test_pKa_acid_data.csv'
output_path = r'C:\work\DrugDiscovery\main_git\XAI_Chem\data\output_EAI\pKa_acid_all_val_smiles'

smiles_and_fgroup_dict = extract_all_smiles_and_fgroups(pKa_acid_csv_path)

if __name__ == '__main__':
    with open(r'C:\work\DrugDiscovery\main_git\XAI_Chem\ml_part\configs\args.pickle', 'rb') as file:
        args =pickle.load(file)
        args['task'] = ['pKa']
    with open(r'C:\work\DrugDiscovery\main_git\XAI_Chem\ml_part\configs\configure.json', 'r') as f:
        exp_config = json.load(f)
    args['device'] = torch.device('cpu')
    args = setup(args)

    logP_model = load_logP_model(exp_config, args)
    amine_model = load_pKa_basic_model(args)
    acid_model = load_pKa_acidic_model(args)

    logP_model.eval()
    amine_model.eval()
    acid_model.eval()

    index = 0
    for smiles, fgroup in tqdm(smiles_and_fgroup_dict.items()):

        bg = convert_smiles_to_dglgraph(smiles, args)
        logP_pred, pKa_acid_pred, pKa_amine_pred = predict(args, logP_model, bg, default_weights=False)
        
        lrp_class = Lrp(logP_model=logP_model, 
                        amine_pKa_model=amine_model,
                        acid_pKa_model=acid_model,
                        smiles=smiles, 
                        functional_group=fgroup, 
                        exp_config=exp_config, 
                        args=args,
                        model_type=ModelType.pKa_acid)

        svg_filename = f'{index}_{smiles}.svg'
        output_svg_path = os.path.join(output_path, svg_filename)
        lrp_class.save_molecule_with_relevances(output_svg_path)
        index += 1
        print(index, smiles, logP_pred, pKa_acid_pred, pKa_amine_pred)

in node features  76


 12%|█▎        | 1/8 [00:00<00:05,  1.17it/s]

1 CCC(F)(F)CC(O)=O tensor([[0.2405, 2.2281]], grad_fn=<AddmmBackward0>) tensor([[3.8902]]) tensor([[-inf]])


 25%|██▌       | 2/8 [00:01<00:05,  1.02it/s]

2 OC(C1C2(CC2)C1(F)F)=O tensor([[0.2727, 2.1228]], grad_fn=<AddmmBackward0>) tensor([[3.2884]]) tensor([[-inf]])


...

6 FC1(F)CC(C(O)=O)CCC1 tensor([[0.5114, 2.4474]], grad_fn=<AddmmBackward0>) tensor([[4.1053]]) tensor([[-inf]])


 88%|████████▊ | 7/8 [00:07<00:01,  1.18s/it]

7 OC(C12OCC(C(F)(F)F)(C2)C1)=O tensor([[0.3609, 2.3385]], grad_fn=<AddmmBackward0>) tensor([[2.9560]]) tensor([[-inf]])


100%|██████████| 8/8 [00:08<00:00,  1.06s/it]

8 OC(C1C2(CC2)C1)=O tensor([[0.3702, 2.0910]], grad_fn=<AddmmBackward0>) tensor([[4.1207]]) tensor([[-inf]])


In [9]:

pKa_acid_csv_path = r'C:\work\DrugDiscovery\main_git\XAI_Chem\data\FGroupData\test_logP_data.csv'
output_path = r'C:\work\DrugDiscovery\main_git\XAI_Chem\data\output_EAI\logP_all_val_smiles'

smiles_and_fgroup_dict = extract_all_smiles_and_fgroups(pKa_acid_csv_path)

if __name__ == '__main__':
    with open(r'C:\work\DrugDiscovery\main_git\XAI_Chem\ml_part\configs\args.pickle', 'rb') as file:
        args =pickle.load(file)
        args['task'] = ['pKa']
    with open(r'C:\work\DrugDiscovery\main_git\XAI_Chem\ml_part\configs\configure.json', 'r') as f:
        exp_config = json.load(f)
    args['device'] = torch.device('cpu')
    args = setup(args)

    logP_model = load_logP_model(exp_config, args)
    amine_model = load_pKa_basic_model(args)
    acid_model = load_pKa_acidic_model(args)

    logP_model.eval()
    amine_model.eval()
    acid_model.eval()

    index = 0
    for smiles, fgroup in tqdm(smiles_and_fgroup_dict.items()):

        bg = convert_smiles_to_dglgraph(smiles, args)
        logP_pred, pKa_acid_pred, pKa_amine_pred = predict(args, logP_model, bg, default_weights=False)
        
        lrp_class = Lrp(logP_model=logP_model, 
                        amine_pKa_model=amine_model,
                        acid_pKa_model=acid_model,
                        smiles=smiles, 
                        functional_group=fgroup, 
                        exp_config=exp_config, 
                        args=args,
                        model_type=ModelType.logP)

        svg_filename = f'{index}_{smiles}.svg'
        output_svg_path = os.path.join(output_path, svg_filename)
        lrp_class.save_molecule_with_relevances(output_svg_path)
        index += 1
        print(index, smiles, logP_pred, pKa_acid_pred, pKa_amine_pred)

in node features  76


  4%|▎         | 1/27 [00:00<00:24,  1.06it/s]

1 NC1CCC1 tensor([[-0.2075,  1.8085]], grad_fn=<AddmmBackward0>) tensor([[12.7214]]) tensor([[10.0417]])


  7%|▋         | 2/27 [00:02<00:27,  1.09s/it]

2 FC1(F)CC1C(O)=O tensor([[0.2480, 1.8182]], grad_fn=<AddmmBackward0>) tensor([[3.3733]]) tensor([[-inf]])


...

25 C[C@@H]1C[C@H]1C(O)=O tensor([[0.3234, 2.0620]], grad_fn=<AddmmBackward0>) tensor([[4.5412]]) tensor([[-inf]])


 96%|█████████▋| 26/27 [00:34<00:01,  1.32s/it]

26 FC1(F)CC(C(O)=O)CCC1 tensor([[0.5114, 2.4474]], grad_fn=<AddmmBackward0>) tensor([[4.1053]]) tensor([[-inf]])


100%|██████████| 27/27 [00:36<00:00,  1.34s/it]

27 CC(F)(F)CCC(O)=O tensor([[0.2840, 1.9114]], grad_fn=<AddmmBackward0>) tensor([[4.1469]]) tensor([[-inf]])


FOR normalization

In [6]:

pKa_acid_csv_path = r'C:\work\DrugDiscovery\main_git\XAI_Chem\data\FGroupData\test_pKa_acid_data.csv'

smiles_and_fgroup_dict = extract_all_smiles_and_fgroups(pKa_acid_csv_path)

if __name__ == '__main__':
    with open(r'C:\work\DrugDiscovery\main_git\XAI_Chem\ml_part\configs\args.pickle', 'rb') as file:
        args =pickle.load(file)
        args['task'] = ['pKa']
    with open(r'C:\work\DrugDiscovery\main_git\XAI_Chem\ml_part\configs\configure.json', 'r') as f:
        exp_config = json.load(f)
    args['device'] = torch.device('cpu')
    args = setup(args)

    logP_model = load_logP_model(exp_config, args)
    amine_model = load_pKa_basic_model(args)
    acid_model = load_pKa_acidic_model(args)

    logP_model.eval()
    amine_model.eval()
    acid_model.eval()

    negative_min_global, negative_max_global = 0, pow(-10, 5)
    positive_min_global, positive_max_global = pow(10, 5), 0
    for smiles, fgroup in tqdm(smiles_and_fgroup_dict.items()):

        negative_min, negative_max = 0, pow(-10, 5)
        positive_min, positive_max = pow(10, 5), 0

        bg = convert_smiles_to_dglgraph(smiles, args)
        logP_pred, pKa_acid_pred, pKa_amine_pred = predict(args, logP_model, bg, default_weights=False)
        
        lrp_class = Lrp(logP_model=logP_model, 
                        amine_pKa_model=amine_model,
                        acid_pKa_model=acid_model,
                        smiles=smiles, 
                        functional_group=fgroup, 
                        exp_config=exp_config, 
                        args=args,
                        model_type=ModelType.logP)

        node_relevances = lrp_class.node_relevances
        edge_relevances = lrp_class.edge_relevances

        node_positive_values = [(value) for value in node_relevances.values() if value >= 0]
        edge_positive_values = [(value) for value in edge_relevances.values() if value >= 0]

        node_negative_values = [(value) for value in node_relevances.values() if value < 0]
        edge_negative_values = [(value) for value in edge_relevances.values() if value < 0]

        negative_min = min(min(node_negative_values), min(edge_negative_values))
        negative_max = max(max(node_negative_values), max(edge_negative_values))

        positive_min = min(min(node_positive_values), min(edge_positive_values))
        positive_max = max(max(node_positive_values), max(edge_positive_values))
        
        negative_min_global = min(negative_min_global, negative_min)
        negative_max_global = min(negative_max_global, negative_max)

        positive_min_global = min(positive_min_global, positive_min)
        positive_max_global = max(positive_max_global, positive_max)



in node features  76


  4%|▎         | 1/27 [00:01<00:32,  1.24s/it]

{0: 0.1979735642671585, 1: 0.06554101407527924, 2: 0.06554100662469864, 3: -0.005515102297067642, 4: 0.10672817379236221}


  7%|▋         | 2/27 [00:02<00:31,  1.27s/it]

{0: 0.2264520227909088, 1: 0.14152777194976807, 2: 0.029188591986894608, 3: -0.19476503133773804, 4: 0.13924385607242584, 5: 0.0758892074227333, 6: 0.14195217192173004, 7: 0.2264520227909088}


...

{0: 0.04504605382680893, 1: 0.09800705313682556, 2: 0.08443775773048401, 3: 0.10892130434513092, 4: 0.14384274184703827, 5: 0.06793485581874847, 6: -0.010737404227256775}


 96%|█████████▋| 26/27 [00:42<00:01,  1.78s/it]

{0: 0.18873661756515503, 1: -0.08342565596103668, 2: -0.05145120993256569, 3: 0.14282602071762085, 4: 0.21491745114326477, 5: 0.02814723551273346, 6: 0.06753905117511749, 7: 0.18873661756515503, 8: 0.12110663205385208, 9: -0.03026546910405159, 10: 0.15943527221679688}


100%|██████████| 27/27 [00:43<00:00,  1.61s/it]

{0: 0.13569460809230804, 1: 0.1467602550983429, 2: 0.15088029205799103, 3: -0.04475916177034378, 4: 0.1506797969341278, 5: 0.049493901431560516, 6: 0.18846048414707184, 7: 0.15088030695915222, 8: 0.14780446887016296}
